In [1]:
import pandas
import numpy as np

In [2]:
li=[]
for i in range(10,20):
    print('{}{}'.format(i,i+1))
    boxscore=pandas.read_csv('./boxscore/gamehunx{}{}.csv'.format(i,i+1),header=None)
    li.append(boxscore)
boxscore = pandas.concat(li, axis=0, ignore_index=True)

1011
1112
1213
1314
1415
1516
1617
1718
1819
1920


In [3]:
boxscore.columns=["CODE","team","GM","Min","PF","DF","2Pm","2Pa","2PP","3Pm","3Pa","3PP","FTm","Fta","FTP","ORB","DRB","TRB","BS","SR","TOV","ST","ASS","PTS", "VALU","OP","OER","ASTO","VIR","extra1","date","extra3","nat"]

In [4]:
boxscore=boxscore.drop(axis=1,labels=['VALU','OP','OER','ASTO','VIR','extra3','GM','CODE','nat'])

In [5]:
f = dict.fromkeys(boxscore.columns, 'sum')
f.update(
    dict.fromkeys(['2PP','3PP','FTP','date'], 'mean'))

In [6]:
result=boxscore.groupby(['extra1','date','team']).agg(f)

In [7]:
result=result.drop(axis=1,labels=['extra1','team','Min','date'])
result.columns

Index(['PF', 'DF', '2Pm', '2Pa', '2PP', '3Pm', '3Pa', '3PP', 'FTm', 'Fta',
       'FTP', 'ORB', 'DRB', 'TRB', 'BS', 'SR', 'TOV', 'ST', 'ASS', 'PTS'],
      dtype='object')

In [8]:
result=result.reset_index()

In [9]:
result=result.sort_values(["date",'extra1'])

In [10]:
result

,extra1,date,team,PF,DF,2Pm,2Pa,2PP,3Pm,3Pa,...,FTP,ORB,DRB,TRB,BS,SR,TOV,ST,ASS,PTS
1240,900,20101009,9005,18,17,23,32,607.666667,12,31,...,541.666667,6,16,22,5,1,9,20,27,97
1241,900,20101009,40051,17,18,22,42,297.090909,3,10,...,382.272727,12,12,24,1,5,14,11,4,67
1244,901,20101009,9007,17,27,23,35,421.545455,6,24,...,431.818182,10,26,36,4,3,11,8,17,86
1245,901,20101009,9013,27,17,26,49,244.333333,5,17,...,229.166667,7,19,26,3,4,11,9,13,77
1248,902,20101009,9003,14,18,26,40,577.750000,14,26,...,494.416667,11,25,36,0,0,12,9,21,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615,1302,20200307,9230,15,16,24,37,316.916667,7,23,...,429.166667,3,19,22,0,0,9,9,20,82
2618,1303,20200307,9002,22,20,20,35,467.083333,8,29,...,347.250000,10,23,33,4,3,15,7,16,74
2619,1303,20200307,9006,20,22,24,41,378.800000,6,23,...,362.500000,8,26,34,3,4,10,6,20,79
2622,1304,20200307,6015,21,20,20,39,279.166667,3,23,...,451.416667,8,25,33,2,0,14,8,16,66


In [11]:
data=pandas.DataFrame(columns=['gameid',"teamid","2PP","3PP","DRB%","ASS%","TOV%","Win%","3PTS/PTS","FTP"])

In [12]:
data['gameid']=result['extra1']
data["teamid"]=result["team"]
data['date']=result['date']
data["2PP"]=result["2PP"]/10
data["3PP"]=result["3PP"]/10
data["FTP"]=result["FTP"]/10
data["3PTS/PTS"]=result["3Pm"]*3/result["PTS"]*100
data["FT/PTS"]=result["FTm"]/result["PTS"]*100
data["Poss"]=0.96*(result['2Pa']+result['3Pa']+0.44*result['Fta']+result['TOV']-result['ORB'])
data["ASS%"]=result['ASS']/(result['2Pa']+result['3Pa']+0.44*result['Fta']+result['TOV']+result['ASS'])*100
data["TOV%"]=result['TOV']/(result['2Pa']+result['3Pa']+0.44*result['Fta']+result['TOV']+result['ASS'])*100
data['OffRating']=100*result['PTS']/data['Poss']
data['DF']=result['DF']

In [13]:
from itertools import tee
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [14]:
lista1=[]
lista2=[]
lista3=[]
lista4=[]
lista5=[]
lista6=[]
lista7=[]
lista8=[]
for (i1, row1), (i2, row2) in pairwise(result.sort_values(['date','extra1']).iterrows()):    
    if(row1['extra1']==row2['extra1']):
        if(row1['PTS']>row2['PTS']):
            lista1.append(100)
            lista1.append(0)
        else:
            lista1.append(0)
            lista1.append(100)
        lista2.append(row1['DRB']/(row2['Fta']-row2['FTm']+row2['2Pa']-row2['2Pm']+row2['3Pa']-row2['3Pm'])*100)
        lista2.append(row2['DRB']/(row1['Fta']-row1['FTm']+row1['2Pa']-row1['2Pm']+row1['3Pa']-row1['3Pm'])*100)
        lista3.append(row2['PTS']/(0.96*(row1['2Pa']+row1['3Pa']+0.44*row1['Fta']+row1['TOV']-row1['ORB']))*100)
        lista3.append(row1['PTS']/(0.96*(row2['2Pa']+row2['3Pa']+0.44*row2['Fta']+row2['TOV']-row2['ORB']))*100)
        lista4.append(row1['ST']/(0.96*(row2['2Pa']+row2['3Pa']+0.44*row2['Fta']+row2['TOV']-row2['ORB']))*100)
        lista4.append(row2['ST']/(0.96*(row1['2Pa']+row1['3Pa']+0.44*row1['Fta']+row1['TOV']-row1['ORB']))*100)
        lista5.append(row1['ORB']/(row1['ORB']+row2['DRB'])*100)
        lista5.append(row2['ORB']/(row2['ORB']+row1['DRB'])*100)
        lista6.append(row1['BS']/(row2['2Pa']+row2['3Pa'])*100)
        lista6.append(row2['BS']/(row1['2Pa']+row1['3Pa'])*100)
        lista7.append(row2['3PP']/10)
        lista7.append(row1['3PP']/10)
        lista8.append(row2['2PP']/10)
        lista8.append(row1['2PP']/10)

In [15]:
data['Win%']=lista1
data['DRB%']=lista2
data['DefRating']=lista3
data['ST%']=lista4
data['ORB%']=lista5
data['BL%']=lista6
data['Opp_3PP']=lista7
data['Opp_2PP']=lista8

In [16]:
data=data.sort_values(['teamid','date','gameid'])
data=data.reset_index()
data=data.drop(axis=1,columns=['index'])

In [17]:
szezon=[]
date=20100830
for j,r in data.iterrows():
    for i in range(0,10):
        if((r['date']>date+i*10000) &(r['date']<date+(i+1)*10000)):
            szezon.append(i)
            break

In [18]:
data['szezon']=szezon

In [21]:
#byszezon=data[~data['teamid'].isin([9221,9233,40051,9218])]
byszezon=data
byszezon=byszezon[byszezon['szezon'].isin([0,1,2,3,4])]

In [22]:
elozoevekatlaga=byszezon.groupby(['teamid']).mean()
elozoevekatlaga=elozoevekatlaga.reset_index()
elozoevekatlaga

,teamid,gameid,2PP,3PP,DRB%,ASS%,TOV%,Win%,3PTS/PTS,FTP,...,Poss,OffRating,DF,DefRating,ST%,ORB%,BL%,Opp_3PP,Opp_2PP,szezon
0,6015,1140.288462,42.595697,21.959240,59.914784,18.156450,10.872723,82.692308,29.474565,34.226799,...,70.382031,124.746865,19.057692,105.911842,15.901183,32.079910,5.620480,16.259889,33.616218,0.500000
1,9001,1120.800000,39.356783,17.987237,64.013993,14.807454,12.710589,71.666667,24.741785,32.012452,...,72.988160,117.650282,19.925000,101.492982,15.509625,32.229702,5.443426,16.076238,33.322914,1.933333
2,9002,1120.475000,40.668429,17.270149,60.407270,15.407434,13.710635,59.166667,23.490759,33.054324,...,72.288320,118.639918,20.708333,113.374905,10.740325,30.200652,3.736210,16.364319,36.046290,1.933333
3,9003,1120.391667,37.847437,19.433118,58.816998,13.922359,11.365979,58.333333,31.058435,28.226679,...,71.236480,118.679581,20.041667,112.731542,12.692971,28.280618,4.012917,15.834923,34.843273,1.933333
4,9004,1119.816667,33.842223,14.381408,61.114329,14.877015,12.804301,53.333333,22.300031,32.562601,...,72.574080,113.675428,20.641667,115.364718,12.482154,27.490029,3.809771,18.331621,38.086919,1.933333
5,9005,1120.333333,34.662875,15.851055,59.097737,13.819774,13.273841,29.166667,27.035320,30.222687,...,69.535040,107.813804,19.666667,120.233740,14.108155,26.818743,2.658429,17.683344,38.988755,1.933333
6,9006,1120.966667,38.265043,18.413283,61.547069,14.747211,12.727951,54.166667,30.450876,31.474288,...,69.016320,115.592828,19.866667,112.446512,11.601659,28.550456,3.436831,17.005899,34.415398,1.933333
7,9007,1120.450000,40.917389,19.548391,62.393214,15.552992,12.194581,75.833333,25.673593,35.882245,...,71.409920,118.629943,19.716667,103.956376,13.004768,32.007042,3.750639,15.889502,33.061896,1.933333
8,9011,1120.558333,33.327924,14.952937,59.471119,12.622033,12.700025,32.500000,24.756136,28.875064,...,71.640000,108.775117,20.525000,118.505682,15.389216,24.862605,3.129533,19.288118,37.963844,1.933333
9,9013,1121.166667,33.078025,14.625654,59.348402,14.631222,13.929214,37.500000,26.227160,33.065955,...,74.097600,110.368489,20.500000,115.517411,11.370803,25.040958,3.112542,16.696311,38.835239,1.933333


In [23]:
data.groupby(['teamid','szezon']).count().groupby('teamid').count()

,gameid,2PP,3PP,DRB%,ASS%,TOV%,Win%,3PTS/PTS,FTP,date,FT/PTS,Poss,OffRating,DF,DefRating,ST%,ORB%,BL%,Opp_3PP,Opp_2PP
teamid,,,,,,,,,,,,,,,,,,,,
6015,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
9001,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9002,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9003,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9004,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9005,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9006,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9007,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9011,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


In [302]:
cnt=1
actual_team=0
index=0
lista=[]
for i,r in data.iterrows():
    adatok=[]
    if(r["teamid"]==actual_team):
        if(r['date']>20150830):
            cnt+=1
            if((actual_team in [40010,9217,9233,40000])&(cnt>5)):
                last_five=data[i-5:i]
                season=data[i-min(cnt,20):i]
                eves=data[i-cnt:i]
                adatok.append(r['gameid'])
                adatok.append(actual_team)
                for c in data.columns:
                    if(c  in ['OffRating', 'ASS%','2PP', '3PP','TOV%','DRB%','ST%','BL%']):
                        adatok.append(last_five[c].mean())
                    if(c  in ['DefRating','Win%','3PTS/PTS', 'FTP','FT/PTS','Poss','Opp_3PP','DF']):
                        adatok.append(season[c].mean())
                    if(c  in ['OffRating','Win%','3PTS/PTS', 'ST%','FT/PTS','ORB%','Opp_2PP','TOV%']):
                        adatok.append(eves[c].mean())
                adatok.append(r['date'])
                lista.append(adatok)
            if(actual_team not in [40010,9217,9233,40000]):
                last_five=data[i-5:i]
                season=data[i-20:i]
                eves=elozoevekatlaga[elozoevekatlaga['teamid']==actual_team]
                adatok.append(r['gameid'])
                adatok.append(actual_team)
                for c in data.columns:
                    if(c  in ['OffRating', 'ASS%','2PP', '3PP','TOV%','DRB%','ST%','BL%']):
                        #print(c,':',last_five[c].mean())
                        adatok.append(last_five[c].mean())
                    if(c  in ['DefRating','Win%','3PTS/PTS', 'FTP','FT/PTS','Poss','Opp_3PP','DF']):
                        #print(c,':',season[c].mean())
                        adatok.append(season[c].mean())
                    if(c  in ['OffRating','Win%','3PTS/PTS', 'ST%','FT/PTS','ORB%','Opp_2PP','TOV%']):
                        #print(c,':',eves[c].mean())
                        adatok.append(eves[c].mean())
                adatok.append(r['date'])
                lista.append(adatok)
    else:
        actual_team=r["teamid"]
        cnt=1
        first=0

In [303]:
len(lista)

1718

In [304]:
data2=pandas.DataFrame(data=lista,columns=['gameid',"teamid",'2PP_g', '3PP_g', 'DRB%_g', 'ASS%_g', 'TOV%_g','TOV%_y', 'Win%_s','Win%_y','3PTS/PTS_s','3PTS/PTS_y', 'FTP_s','FT/PTS_s','FT/PTS_y', 'Poss_s', 'OffRating_g','OffRating_y', 'DF_s','DefRating_s','ST%_g','ST%_y','ORB%_y','BL%_g','Opp_3PP_s','Opp_2PP_y','date'])

In [305]:
tabla=[]
for (i1, row1), (i2, row2) in pairwise(data2.sort_values(['date','gameid']).iterrows()):
    sor=[]
    if(row1['gameid']==row2['gameid']):
        for c in data2.columns:
            sor.append(row1[c])
        for c in data2.columns:
            if(c not in ['gameid','date']):
                sor.append(row2[c])
        tabla.append(sor)

In [265]:
len(tabla)

729

In [358]:
test=pandas.DataFrame(data=tabla,columns=['gameid',"teamid_1",'2PP_g_1', '3PP_g_1', 'DRB%_g_1', 'ASS%_g_1', 'TOV%_g_1','TOV%_y_1', 'Win%_s_1','Win%_y_1','3PTS/PTS_s_1','3PTS/PTS_y_1', 'FTP_s_1','FT/PTS_s_1','FT/PTS_y_1', 'Poss_s_1', 'OffRating_g_1','OffRating_y_1', 'DF_s_1','DefRating_s_1','ST%_g_1','ST%_y_1','ORB%_y_1','BL%_g_1','Opp_3PP_s_1','Opp_2PP_y_1','date',"teamid_2",'2PP_g_2', '3PP_g_2', 'DRB%_g_2', 'ASS%_g_2', 'TOV%_g_2','TOV%_y_2', 'Win%_s_2','Win%_y_2','3PTS/PTS_s_2','3PTS/PTS_y_2', 'FTP_s_2','FT/PTS_s_2','FT/PTS_y_2', 'Poss_s_2', 'OffRating_g_2','OffRating_y_2', 'DF_s_2','DefRating_s_2','ST%_g_2','ST%_y_2','ORB%_y_2','BL%_g_2','Opp_3PP_s_2','Opp_2PP_y_2'])

In [50]:
test.to_csv('test5.csv',index=False)

In [163]:
#legközelebb elég csak beolvasni

In [359]:
results=pandas.read_csv('allgames.csv')

In [342]:
test=pandas.read_csv('test.csv')

In [360]:
merge=pandas.merge(results,test,on=['date',"gameid"],how='inner')

In [361]:
merge=merge.drop(axis=1,columns=['team1','team2'])

In [362]:
merge

,gameid,date,won,teamid_1,2PP_g_1,3PP_g_1,DRB%_g_1,ASS%_g_1,TOV%_g_1,TOV%_y_1,...,OffRating_g_2,OffRating_y_2,DF_s_2,DefRating_s_2,ST%_g_2,ST%_y_2,ORB%_y_2,BL%_g_2,Opp_3PP_s_2,Opp_2PP_y_2
0,1.0,20151002.0,9002.0,9002.0,33.308333,20.745000,57.382955,12.505598,13.431058,13.710635,...,109.256291,112.566533,20.85,99.198944,22.221129,15.519533,29.169716,4.198585,15.611591,33.644362
1,2.0,20151002.0,9003.0,9003.0,35.587727,25.207576,63.195352,15.796283,8.554508,11.365979,...,116.655336,110.368489,21.75,118.125831,7.399379,11.370803,25.040958,2.341774,16.447583,38.835239
2,3.0,20151003.0,9007.0,9004.0,40.517727,11.299394,64.195754,18.430628,10.994301,12.804301,...,112.494840,118.629943,20.55,97.297803,10.802425,13.004768,32.007042,2.464621,17.834143,33.061896
3,4.0,20151003.0,9011.0,9011.0,38.310000,13.156818,63.950926,12.680857,15.176753,12.700025,...,108.375657,102.141270,22.75,118.839116,9.320800,8.523174,27.558333,1.751748,15.892879,36.522273
4,5.0,20151003.0,9006.0,9006.0,40.456667,16.730000,59.965955,13.914138,14.439003,12.727951,...,110.058565,112.299822,20.95,110.070585,12.084063,12.944563,26.604648,2.407755,16.315030,36.201250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,1150.0,20190406.0,9213.0,9005.0,33.906667,24.131667,60.342422,14.038413,11.944251,13.273841,...,110.126488,112.299822,20.00,110.058647,9.294047,12.944563,26.604648,3.500005,16.640417,36.201250
535,1151.0,20190406.0,9003.0,9003.0,39.928333,24.350000,55.253400,16.170592,9.003446,11.365979,...,112.978479,118.629943,19.55,109.768089,6.541256,13.004768,32.007042,5.251906,18.596174,33.061896
536,1152.0,20190406.0,9800.0,9004.0,38.610152,15.927576,61.304213,15.918888,12.366808,12.804301,...,110.301848,112.566533,19.90,114.180400,7.484431,15.519533,29.169716,2.582279,17.740265,33.644362
537,1153.0,20190406.0,9006.0,9006.0,32.967242,21.438242,57.097886,16.555643,10.153468,12.727951,...,112.311678,110.368489,20.40,113.221581,8.541584,11.370803,25.040958,2.808089,16.730152,38.835239


In [363]:
w=[]
for i,r in merge.iterrows():
    if(r['won']==r['teamid_1']):
        w.append(1)
    else:
        w.append(2)
merge['won']=w

In [364]:
merge=merge.drop(axis=1,columns=['teamid_1','teamid_2','gameid'])

In [365]:
labels=np.array(merge['won'])

In [366]:
merge=merge.drop(axis=1,columns=['date','won'])

In [367]:
merge = np.array(merge)

In [40]:
from sklearn.model_selection import train_test_split

In [368]:
train_data, test_data, train_labels, test_labels = train_test_split(merge, labels, test_size = 0.1, random_state = 42)

In [369]:
print('Training Features Shape:', train_data.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_data.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (485, 48)
Training Labels Shape: (485,)
Testing Features Shape: (54, 48)
Testing Labels Shape: (54,)


In [60]:
from sklearn.ensemble import RandomForestClassifier

In [355]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [370]:
rf.fit(train_data, train_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [63]:
rf.score(test_data,test_labels)

0.7037037037037037

In [191]:
predictions = rf.predict(test_data)

In [316]:
data2[data2['date']>20190830].groupby('teamid').last()

,gameid,2PP_g,3PP_g,DRB%_g,ASS%_g,TOV%_g,TOV%_y,Win%_s,Win%_y,3PTS/PTS_s,...,OffRating_y,DF_s,DefRating_s,ST%_g,ST%_y,ORB%_y,BL%_g,Opp_3PP_s,Opp_2PP_y,date
teamid,,,,,,,,,,,,,,,,,,,,,
6015.0,1304.0,35.796026,16.485128,56.805978,15.785426,15.068993,10.872723,55.0,82.692308,35.362156,...,124.746865,17.75,107.693604,9.937087,15.901183,32.079910,3.728600,14.695274,33.616218,20200307.0
9001.0,1302.0,33.110000,12.883333,68.263126,16.005310,11.836462,12.710589,55.0,71.666667,31.487074,...,117.650282,19.25,118.675677,4.870876,15.509625,32.229702,0.571429,20.449576,33.322914,20200307.0
9002.0,1303.0,45.968333,27.191667,64.035213,19.551544,8.617553,13.710635,90.0,59.166667,31.633232,...,118.639918,19.70,99.960571,11.541064,10.740325,30.200652,3.323455,15.000530,36.046290,20200307.0
9003.0,1301.0,36.827727,14.384242,59.330137,17.147743,11.224736,11.365979,80.0,58.333333,28.077506,...,118.679581,19.60,106.379076,11.305011,12.692971,28.280618,5.209918,16.629455,34.843273,20200307.0
9004.0,1299.0,35.238788,15.966212,58.526535,15.640860,9.649453,12.804301,35.0,53.333333,31.310541,...,113.675428,20.90,113.164280,8.630611,12.482154,27.490029,4.808566,18.794115,38.086919,20200307.0
9005.0,1298.0,39.751667,22.976667,51.996859,17.039454,10.569098,13.273841,50.0,29.166667,29.299157,...,107.813804,20.75,120.096062,9.349073,14.108155,26.818743,2.723255,20.181652,38.988755,20200304.0
9006.0,1303.0,31.656545,11.796000,55.952298,14.108715,9.382064,12.727951,40.0,54.166667,22.653173,...,115.592828,20.95,110.147101,10.372888,11.601659,28.550456,1.772374,15.891712,34.415398,20200307.0
9007.0,1299.0,41.824727,22.628909,58.079064,15.032882,13.180898,12.194581,60.0,75.833333,32.642592,...,118.629943,19.10,110.258354,9.298179,13.004768,32.007042,7.477491,18.169265,33.061896,20200307.0
9013.0,1300.0,31.151667,13.713333,59.518149,17.655200,11.703578,13.929214,20.0,37.500000,31.975634,...,110.368489,20.30,126.441291,9.570830,11.370803,25.040958,1.554198,20.441023,38.835239,20200307.0


In [372]:
csapatok2020=[9004,9213,9013,9005,9800,40000,40010,9007,9001,9003,9002,9230,9006,6015]
len(csapatok2020)

14

In [378]:
boxscore1920=pandas.read_csv('./boxscore/gamehunx1920.csv',header=None)
boxscore1920.columns=["CODE","team","GM","Min","PF","DF","2Pm","2Pa","2PP","3Pm","3Pa","3PP","FTm","Fta","FTP","ORB","DRB","TRB","BS","SR","TOV","ST","ASS","PTS", "VALU","OP","OER","ASTO","VIR","extra1","date","extra3","nat"]

In [374]:
summa=pandas.DataFrame()

In [379]:
result=boxscore1920.groupby(['extra1','date','team']).agg(f)
result=result.drop(axis=1,labels=['extra1','team','Min','date'])
result=result.reset_index()
result=result.sort_values(["date",'extra1'])

In [380]:
summa['gameid']=result['extra1']
summa["teamid"]=result["team"]
summa['date']=result['date']
summa["2PP"]=result["2PP"]/10
summa["3PP"]=result["3PP"]/10
summa["FTP"]=result["FTP"]/10
summa["3PTS/PTS"]=result["3Pm"]*3/result["PTS"]*100
summa["FT/PTS"]=result["FTm"]/result["PTS"]*100
summa["Poss"]=0.96*(result['2Pa']+result['3Pa']+0.44*result['Fta']+result['TOV']-result['ORB'])
summa["ASS%"]=result['ASS']/(result['2Pa']+result['3Pa']+0.44*result['Fta']+result['TOV']+result['ASS'])*100
summa["TOV%"]=result['TOV']/(result['2Pa']+result['3Pa']+0.44*result['Fta']+result['TOV']+result['ASS'])*100
summa['OffRating']=100*result['PTS']/summa['Poss']
summa['DF']=result['DF']

In [381]:
lista1=[]
lista2=[]
lista3=[]
lista4=[]
lista5=[]
lista6=[]
lista7=[]
lista8=[]
for (i1, row1), (i2, row2) in pairwise(result.sort_values(['date','extra1']).iterrows()):    
    if(row1['extra1']==row2['extra1']):
        if(row1['PTS']>row2['PTS']):
            lista1.append(100)
            lista1.append(0)
        else:
            lista1.append(0)
            lista1.append(100)
        lista2.append(row1['DRB']/(row2['Fta']-row2['FTm']+row2['2Pa']-row2['2Pm']+row2['3Pa']-row2['3Pm'])*100)
        lista2.append(row2['DRB']/(row1['Fta']-row1['FTm']+row1['2Pa']-row1['2Pm']+row1['3Pa']-row1['3Pm'])*100)
        lista3.append(row2['PTS']/(0.96*(row1['2Pa']+row1['3Pa']+0.44*row1['Fta']+row1['TOV']-row1['ORB']))*100)
        lista3.append(row1['PTS']/(0.96*(row2['2Pa']+row2['3Pa']+0.44*row2['Fta']+row2['TOV']-row2['ORB']))*100)
        lista4.append(row1['ST']/(0.96*(row2['2Pa']+row2['3Pa']+0.44*row2['Fta']+row2['TOV']-row2['ORB']))*100)
        lista4.append(row2['ST']/(0.96*(row1['2Pa']+row1['3Pa']+0.44*row1['Fta']+row1['TOV']-row1['ORB']))*100)
        lista5.append(row1['ORB']/(row1['ORB']+row2['DRB'])*100)
        lista5.append(row2['ORB']/(row2['ORB']+row1['DRB'])*100)
        lista6.append(row1['BS']/(row2['2Pa']+row2['3Pa'])*100)
        lista6.append(row2['BS']/(row1['2Pa']+row1['3Pa'])*100)
        lista7.append(row2['3PP']/10)
        lista7.append(row1['3PP']/10)
        lista8.append(row2['2PP']/10)
        lista8.append(row1['2PP']/10)

In [382]:
summa['Win%']=lista1
summa['DRB%']=lista2
summa['DefRating']=lista3
summa['ST%']=lista4
summa['ORB%']=lista5
summa['BL%']=lista6
summa['Opp_3PP']=lista7
summa['Opp_2PP']=lista8

In [386]:
season1920=summa.groupby('teamid').mean()
season1920=season1920.drop(axis=1,columns=['gameid','date'])
season1920

,2PP,3PP,FTP,3PTS/PTS,FT/PTS,Poss,ASS%,TOV%,OffRating,DF,Win%,DRB%,DefRating,ST%,ORB%,BL%,Opp_3PP,Opp_2PP
teamid,,,,,,,,,,,,,,,,,,
6015,34.648635,16.614600,26.000723,34.714860,14.570856,71.988480,16.661041,12.736375,111.344437,17.950000,55.000000,56.463128,106.908566,11.092006,27.372956,3.507710,14.054857,36.334645
9001,39.023750,17.646667,35.906250,31.583184,17.657615,73.703040,18.031004,10.997536,120.385358,18.950000,55.000000,63.151823,119.787354,7.707412,30.754152,2.144880,20.564159,35.382545
9002,40.645833,20.932500,39.473333,31.641951,16.175750,72.522240,16.850121,10.601887,118.667864,19.900000,85.000000,63.901465,100.641161,12.551927,31.190089,4.374235,15.581530,31.834265
9003,40.136477,16.953636,33.462917,27.255847,15.770014,70.944000,16.422948,11.013411,119.010327,19.700000,80.000000,62.400970,106.212387,12.547742,29.712761,4.535974,17.054038,34.587000
9004,33.496140,16.839971,33.720202,31.138244,18.297868,73.424457,15.520818,11.777097,112.225313,20.761905,33.333333,64.251222,113.592372,9.361433,28.384192,3.624589,18.713840,37.082986
9005,34.361538,17.392509,36.728881,29.455971,18.330365,74.125440,14.473132,11.822123,118.496512,21.400000,50.000000,56.202280,119.138071,8.537114,23.492303,4.520970,19.820242,37.252576
9006,32.313947,13.964114,35.485136,22.778900,20.189022,73.355520,14.668228,12.070647,108.105119,21.150000,40.000000,59.461854,110.749358,10.298576,26.154113,2.107840,16.221485,36.086682
9007,35.374106,22.315591,33.482727,32.400984,15.373086,74.722560,15.253965,11.111002,115.286866,19.450000,65.000000,62.289882,108.537146,10.312670,29.348992,5.855721,17.274265,33.401182
9013,27.888528,15.889141,27.305483,32.284487,15.116187,70.981486,15.756126,11.493698,112.831349,20.142857,19.047619,58.231981,126.650500,9.944427,25.619961,1.196653,21.358550,40.672367


In [388]:
five=summa.groupby("teamid").tail(5).groupby("teamid").mean()
five=five.drop(axis=1,columns=['gameid','date'])
five

,2PP,3PP,FTP,3PTS/PTS,FT/PTS,Poss,ASS%,TOV%,OffRating,DF,Win%,DRB%,DefRating,ST%,ORB%,BL%,Opp_3PP,Opp_2PP
teamid,,,,,,,,,,,,,,,,,,
6015,31.561026,14.040128,27.134103,25.942221,18.238527,73.94304,16.677847,15.121488,103.219746,20.4,20.0,55.175543,108.109894,9.210178,27.596547,2.472190,14.432273,35.983939
9001,33.676667,14.695000,36.505000,29.610959,20.782942,71.05536,17.206999,10.680605,116.984283,20.2,60.0,72.232823,120.288728,5.582782,24.679372,0.571429,20.820000,34.215000
9002,43.810000,24.556667,36.113333,32.940969,13.246278,71.43936,18.105174,9.955760,117.701009,19.8,60.0,65.769971,103.574031,10.939294,25.860153,3.283133,14.382333,35.092667
9003,38.116515,12.472727,29.571970,26.222205,14.544558,69.85728,17.757110,9.933324,121.597048,19.8,80.0,61.970469,113.107374,14.020398,35.031997,3.888968,22.180152,37.263030
9004,33.880455,15.674545,34.718182,30.789601,17.534262,76.43136,15.278263,10.118456,108.414928,21.8,20.0,60.802495,117.350114,8.567409,28.779169,5.111596,13.132098,35.935571
9005,35.871667,20.775000,42.411667,29.695467,21.324179,77.42208,15.716549,10.557196,129.769513,20.6,80.0,58.336145,118.066588,10.035655,28.747185,2.723255,17.819939,43.037909
9006,35.250727,14.899091,45.748182,22.391284,22.808139,76.20096,15.428388,9.682015,112.894762,23.8,60.0,57.788911,112.320735,10.107606,24.686500,2.352731,16.136091,36.266424
9007,40.632727,19.512576,36.415303,30.133977,15.863723,73.72800,14.348815,12.362811,116.352014,21.6,40.0,60.936207,115.917830,10.158843,37.038799,7.229435,21.213333,35.035606
9013,27.501667,15.353333,20.868333,32.446738,12.324380,70.30272,15.743594,9.900317,112.814042,18.8,20.0,59.998918,121.788753,9.977266,26.613153,1.231618,23.720152,32.428939


In [390]:
years=data[data['szezon'].isin([5,6,7,8,9])].groupby(['teamid']).mean()
years=years.drop(axis=1,columns=['gameid','szezon'])
years

,2PP,3PP,DRB%,ASS%,TOV%,Win%,3PTS/PTS,FTP,date,FT/PTS,Poss,OffRating,DF,DefRating,ST%,ORB%,BL%,Opp_3PP,Opp_2PP
teamid,,,,,,,,,,,,,,,,,,,
6015,38.548911,18.207416,62.464608,16.998771,11.862271,71.774194,31.877290,32.720007,2.017470e+07,15.953075,72.368826,114.938161,20.435484,104.707459,10.471149,27.978308,3.339026,16.698068,32.964518
9001,38.833229,19.084489,63.939679,16.818232,11.616656,64.516129,29.424492,32.352364,2.017470e+07,15.586649,73.318916,115.318789,19.911290,107.246355,10.821812,29.427154,3.884725,18.004157,32.822231
9002,35.655519,17.961783,61.789198,17.442936,9.907258,66.935484,33.554609,31.407142,2.017470e+07,15.515742,71.492129,117.064666,20.161290,108.540855,10.546827,29.623805,3.106590,18.250481,32.508684
9003,37.801299,19.223262,62.452272,15.984115,11.519823,72.580645,31.314146,32.652325,2.017477e+07,15.657221,72.059458,118.276078,19.943548,107.382744,11.708527,29.288836,3.339144,17.254345,33.679424
9004,34.155149,15.672923,62.479398,16.327333,12.550007,46.400000,27.480407,30.456071,2.017490e+07,15.957031,72.306586,112.297053,19.240000,113.238448,8.670302,29.067226,3.864941,17.577902,34.673725
9005,33.547558,18.784910,61.440436,16.188606,11.421735,57.258065,34.022024,30.239050,2.017470e+07,15.256624,72.113652,116.010840,19.346774,113.375665,9.140242,25.928370,4.120590,17.903427,34.266751
9006,32.775710,18.091708,60.552365,15.304516,11.960701,43.548387,28.483795,31.718624,2.017477e+07,17.968125,71.331406,111.518974,20.322581,111.642128,9.663304,27.282168,3.025053,16.975518,34.646571
9007,34.574586,20.185330,62.657604,16.132171,11.911767,54.032258,33.402782,31.996356,2.017470e+07,15.628935,72.640413,113.462205,19.338710,111.188833,9.715355,28.896494,3.537478,17.596543,33.465517
9011,34.618834,16.646020,55.240100,14.953237,15.311981,19.230769,29.581290,25.039744,2.015601e+07,13.188888,72.141785,105.748761,17.423077,121.114882,10.994407,27.279917,2.772858,19.247471,36.505256


In [416]:
tabla=[]
for t in csapatok2020:
    for cs in csapatok2020:
        adatok=[]
        if(t!=cs):
            #adatok.append(t)
            for c in data.columns:
                if(c  in ['OffRating', 'ASS%','2PP', '3PP','TOV%','DRB%','ST%','BL%']):
                    adatok.append(five.groupby('teamid').get_group(t)[c].mean())
                if(c  in ['DefRating','Win%','3PTS/PTS', 'FTP','FT/PTS','Poss','Opp_3PP','DF']):
                    adatok.append(season1920.groupby('teamid').get_group(t)[c].mean())
                if(c  in ['OffRating','Win%','3PTS/PTS', 'ST%','FT/PTS','ORB%','Opp_2PP','TOV%']):
                    adatok.append(years.groupby('teamid').get_group(t)[c].mean())
            #adatok.append(cs)
            for c in data.columns:
                if(c  in ['OffRating', 'ASS%','2PP', '3PP','TOV%','DRB%','ST%','BL%']):
                    adatok.append(five.groupby('teamid').get_group(cs)[c].mean())
                if(c  in ['DefRating','Win%','3PTS/PTS', 'FTP','FT/PTS','Poss','Opp_3PP','DF']):
                    adatok.append(season1920.groupby('teamid').get_group(cs)[c].mean())
                if(c  in ['OffRating','Win%','3PTS/PTS', 'ST%','FT/PTS','ORB%','Opp_2PP','TOV%']):
                    adatok.append(years.groupby('teamid').get_group(cs)[c].mean())
            tabla.append(adatok)
            

In [418]:
predictions=rf.predict(np.array(tabla))

In [426]:
j=0
nyert=[]
for t in csapatok2020:
    for cs in csapatok2020:
        if(t!=cs):
            if(predictions[j]==1):
                nyert.append(t)
            else:
                nyert.append(cs)
            j+=1

In [427]:
for cs in csapatok2020:
    print(cs, nyert.count(cs))

9004 12
9213 13
9013 9
9005 14
9800 6
40000 16
40010 12
9007 25
9001 1
9003 21
9002 19
9230 10
9006 10
6015 14
